# Systemd

## Systemdとは

* Linuxの起動処理やシステム管理を行う仕組み
* Linuxの起動処理4段階の中での位置づけ
    1. 電源投入によりBIOSが起動する
    1. BIOSからブートローダーが呼び出される
    1. ブートローダーがLinuxカーネルを起動する
    1. Linuxカーネルがinitプロセス(PID 1)を起動する
        * initプロセス
            * Sysvinit : Linuxで古くから使われていた
            * Upstart : Ununtuで一時使われていた
            * <font color="red">Systemd</font> : 現在多くのLinuxでデフォルトinitシステムとして採用されている
* 特徴
    * シェルスクリプトではない
        * <font color="red">サービスを「Unit」という単位で管理</font>し、設定ファイルとして持つ
            * 処理を細分化でき個別に実効することが可能
        * 処理ごとの依存関係を明確にすることが出来る
        * 並列での実行も可能
            * Aの処理の後にBとCを並列実行するなど
    * プロセス起動が柔軟
        * 様々なトリガーを契機にプロセスの起動が可能
            * タイマー
            * ソケット通信検出
            * ファイルシステムのマウント
            * etc...
    * PIDではなくcgroupによってプロセスを管理する
        * SysvinitではサービスをPIDによって管理していた
            * プロセスが2回forkすると親プロセスと孫プロセスの直接的な関係性は切れる
            * cgroupの場合は複数回forkが発生しても管理下における
        * cgroupはLinux以外では使えない
            * Linuxカーネルの機能であるため

### Unitタイプ

* .service
    * 指定のバイナリを実行する
* .socket
    * systemdがSocketをListenして、接続があるとプロセスに受け渡す
    * xinetdの代替的な機能
* .target 
    * 複数のユニットをまとめるために使用する
* .device 
    * udevから通知されたデバイスを表す
* .snapshot 
    * ある時点のユニットの状態
* .path 
    * 指定のファイルが生成されると指定されたサービスを起動する
* .mount
    * 指定のファイルシステムをマウントする
* .automount 
    * オートマウント処理を実施する
    * automountdの代替的な機能

### 動作モード

* initの欄レベルに相当する
* Unixタイプの「target」で表す
* systemctlのisolateコマンドで切り替える
    * 「rescue.target」「emergency.target」はisolateをつけずにモードを切り替えるとログインしているユーザにメッセージを送信する
    * それ以外の動作はほぼ同じ
* ターゲットと動作モード
    * rescue.target : レスキューモード
        * どうしても終了しないプロセスがあるなどの状況でシステムを修復する場合に使う
        * シングルユーザーモード(init 1)とほぼ同等
            * システムによってはネットワークシステムを使う・使わないなどの違いがある
    * emergency.target : 緊急モード
        * レスキューモードでも入れない場合に使う
    * multi-user.target : CUIモード
    * graphical.target : GUIモード
    
コマンド例

```
### GUIモードにする
$ sudo systemctl isolate graphical.target

### デフォルト(起動時)のモードにする
$ sudo systemctl default

### レスキューモードにする
$ sudo systemctl rescue
もしくは
$ sudo systemctl isolate rescue.target

### 緊急モードにする
$ sudo systemctl emergency
もしくは
$ sudo systemctl isolate emergency.target

### デフォルトをGUIモードにする
$ sudo systemctl set-default graphical.target

### initのランレベル(ここではランレベル3)での指定も可能
$ systemctl isolate runlevel3.target

### 対応
$ ls -l /usr/lib/systemd/system/runlevel*.target
================================================
lrwxrwxrwx 1 root root 15  4月 21 13:47 /usr/lib/systemd/system/runlevel0.target -> poweroff.target
lrwxrwxrwx 1 root root 13  4月 21 13:47 /usr/lib/systemd/system/runlevel1.target -> rescue.target
lrwxrwxrwx 1 root root 17  4月 21 13:47 /usr/lib/systemd/system/runlevel2.target -> multi-user.target
lrwxrwxrwx 1 root root 17  4月 21 13:47 /usr/lib/systemd/system/runlevel3.target -> multi-user.target
lrwxrwxrwx 1 root root 17  4月 21 13:47 /usr/lib/systemd/system/runlevel4.target -> multi-user.target
lrwxrwxrwx 1 root root 16  4月 21 13:47 /usr/lib/systemd/system/runlevel5.target -> graphical.target
lrwxrwxrwx 1 root root 13  4月 21 13:47 /usr/lib/systemd/system/runlevel6.target -> reboot.target
================================================
```

### @つきサービス

* 複数インスタンスを作成するときに使うテンプレート
* tomcatの例
    * 別ポートでもう1個tomcatを立ち上げたい時
        1. /etc/systemd/system にtomcat@.service を複製
        1. systemctl daemon-reloadを実行
        1. systemctl enable tomcat@inst2.serviceを実行
            * 別インスタンスの設定ができる
        1. /var/lib/tomcats/inst2 に別インスタンスの環境を作る
        1. systemctl start tomcat@inst2.service で起動する
    * tomcatは「tomcat.service」も「tomcat@.service」も中身はほぼ同じ

## サービス管理

### 有効化されているUnitの一覧表示(list-units)

```
### 表示例
$ systemctl list-units
================================================
UNIT         LOAD    ACTIVE SUB       DESCRIPTION
...
sshd.service loaded  active running   OpenSSH server daemon
...
================================================
```

In [4]:
%%bash

systemctl list-units --no-pager | sed -e 's/                                                   //'

  UNIT                                              LOAD   ACTIVE SUB       
  proc-sys-fs-binfmt_misc.automount                 loaded active running   Arbitrary Executable File Formats File System Automount Point
  sys-devices-pci0000:00-0000:00:07.1-ata2-host2-target2:0:0-2:0:0:0-block-sr0.device                  loaded active plugged   VMware_Virtual_IDE_CDROM_Drive
  sys-devices-pci0000:00-0000:00:15.0-0000:03:00.0-host0-target0:0:0-0:0:0:0-block-sda-sda1.device     loaded active plugged   Virtual_disk 1
  sys-devices-pci0000:00-0000:00:15.0-0000:03:00.0-host0-target0:0:0-0:0:0:0-block-sda-sda2.device     loaded active plugged   LVM PV KNtUzQ-9EEr-26Qy-883S-TNeX-T1f5-b9Gw8F on /dev/sda2 2
  sys-devices-pci0000:00-0000:00:15.0-0000:03:00.0-host0-target0:0:0-0:0:0:0-block-sda.device          loaded active plugged   Virtual_disk
  sys-devices-pci0000:00-0000:00:16.0-0000:0b:00.0-net-eno16780032.device                              loaded active plugged   VMXNET3 Ethernet Controller
  

### インストールされているUnitファイルの一覧表示(list-unit-files)

```
### 表示例
$ systemctl list-unit-files --no-pager | grep sshd
================================================
UNIT FILE                                     STATE
...
sshd-keygen.service                           static
sshd.service                                  enabled
sshd@.service                                 static
sshd.socket                                   disabled
================================================

### serviceのみ表示する場合は -t serviceオプションをつける
$ systemctl list-unit-files -t service --no-pager | grep sshd
================================================
UNIT FILE                                     STATE
...
sshd-keygen.service                           static
sshd.service                                  enabled
sshd@.service                                 static
================================================
```

* STATE
    * enabled
        * 有効になっている
    * disabled
        * 無効になっている
    * static
        * 有効化／無効化の対象外
            * UnitファイルにWantedBy句がそもそも指定されていない
        * 他のユニットに依存するもの
            * これ単体で有効/無効が設定できない

* Unitファイルの実態
    * /usr/lib/systemd/system
        * RPMから直接ファイルが提供される場所
        * システムのデフォルト設定
        * <font color="red">ここのファイルは基本的に直接編集するべきではない</font>
            * カスタマイズは /etc/systemd/system にコピーして実施
    * /etc/systemd/system 
        * ここに同名のファイルを配置するとデフォルト設定より優先される

In [26]:
%%bash

echo "### sshdのシステムのデフォルト設定"
echo '$ ls -lh /usr/lib/systemd/system/*sshd*'
echo "================================================"
ls -lh /usr/lib/systemd/system/*sshd*
echo "================================================"
echo ""

echo "### /etc/systemd/system 配下"
echo '$ ls -lh /etc/systemd/system | sed -e "s/-> /\n-> /"'
echo "================================================"
ls -lh /etc/systemd/system | sed -e "s/-> /\n  -> /"
echo "================================================"
echo ""

echo "### /etc/systemd/system/multi-user.target.wants 配下にsshd.serviceのsymlink"
echo '$ ls -lh /etc/systemd/system/multi-user.target.wants/sshd.service | sed -e "s/-> /\n-> /"'
echo "================================================"
ls -lh /etc/systemd/system/multi-user.target.wants/sshd.service | sed -e "s/-> /\n  -> /"
echo "================================================"
echo ""

echo "### multi-user.targetは有効になっている"
echo '$ systemctl list-unit-files --no-pager | grep multi-user.target'
echo "================================================"
systemctl list-unit-files --no-pager | grep multi-user.target
echo "================================================"
echo "※ つまり、CUIモードではsshd.serviceが起動する"


### sshdのシステムのデフォルト設定
$ ls -lh /usr/lib/systemd/system/*sshd*
-rw-r--r--. 1 root root 313  4月 12 23:05 /usr/lib/systemd/system/sshd-keygen.service
-rw-r--r--. 1 root root 402  4月 12 23:05 /usr/lib/systemd/system/sshd.service
-rw-r--r--. 1 root root 181  4月 12 23:05 /usr/lib/systemd/system/sshd.socket
-rw-r--r--. 1 root root 260  4月 12 23:05 /usr/lib/systemd/system/sshd@.service

### /etc/systemd/system 配下
$ ls -lh /etc/systemd/system | sed -e "s/-> /\n-> /"
合計 4.0K
drwxr-xr-x. 2 root root   30  7月  3 14:27 basic.target.wants
lrwxrwxrwx. 1 root root   46  7月  3 14:16 dbus-org.freedesktop.NetworkManager.service 
  -> /usr/lib/systemd/system/NetworkManager.service
lrwxrwxrwx. 1 root root   57  7月  3 14:16 dbus-org.freedesktop.nm-dispatcher.service 
  -> /usr/lib/systemd/system/NetworkManager-dispatcher.service
lrwxrwxrwx. 1 root root   37  7月  3 14:18 default.target 
  -> /lib/systemd/system/multi-user.target
drwxr-xr-x. 2 root root   85  7月  3 14:16 default.target.wants
drwxr-xr-x. 2 roo

## Unitファイル

### Unitファイル構成

* Unitセクション
    * ユニットファイルの共通的な設定
        * Unitの依存関係/順序関係など
        * Unitのタイプに依存しない設定項目
    * Description
        * ユニットの説明
    * After/Before
        * 指定したユニットの実行順番の依存関係を設定
            * 「After=bar.service」
                * そのサービスはbar.serviceの起動処理が終わるまで起動しない
    * Requires/Wants
        * <font color="red">このUnitと同時に起動されるべきUnitを指定</font>
            * どちらが先に起動するかは限定していない
        * ユニットの依存関係の定義
            * 「Wants=bar.service」
                * bar.serviceの起動が失敗してもそのサービスを起動処理をする
                    * bar.serviceが無くても起動するサービスであれば起動はする
            * 「Requires=bar.service」
                * bar.serviceの起動が失敗したらそのサービスは起動しない
        * 設定以外の指定方法
            1. /etc/systemd/systemに移動
            1. 「<設定ファイル名>.reqruires」および「<設定ファイル名>.wants」というディレクトリを作成
            1. その中に他のUnitの設定ファイルへのシンボリックリンクを作成する
* Serviceセクション
    * そのサービスを実行するための設定
    * Type
        * 「ExecStart」で指定したコマンドでサービスのプロセスを起動した際に、起動完了をどのように判定するかを指定
            * simple
                * コマンドを実行した時点で起動完了と判断する
                    * フォアグラウンドで実行を継続する場合
                * 指定が無い場合のデフォルト
            * forking
                * 実行したコマンドが終了した時点で起動完了と判断する
                    * 子プロセスをフォークしてバックグラウンドにまわして、最初のコマンド自体は終了する場合
            * oneshot
                * 「ExecStart」で指定したコマンドを実行して、それが完了したタイミングで起動完了（かつサービス終了）と判断する
                    * デーモン型のプロセスではなく、一度だけ指定のコマンドを実行する場合
            * dbus
                * フォアグラウンドで実行するタイプの中でもD-Busを利用するサービス
                * 「BusName」にそのサービスのbus接続名を指定
                    * systemdは、D-Bus上にこの接続名の接続が現れたタイミングでサービスが起動したものと判定
    * ExecStart
        * サービス起動コマンド
    * ExecReload
        * サービスリロードコマンド
            * Apacheのgracefulはこれに割り当てられている
    * ExecStop
        * サービス停止コマンド
    * ExecStartPre/ExecStartPost
        * サービス起動前後の追加コマンド
            * サービス起動判定には関連させたくないコマンドを記載
    * ExecStopPost
        * サービス停止後に実行するコマンド
        * サービスが異常停止した際にも実行される
    * EnvironmentFile
        * 環境変数を読み込むファイル
    * PIDFile
        * fork型サービスのメインプロセスのPIDファイル
            * フォークの形式によってどのプロセスがメインプロセスとなるかが変わる
            * systemdはメインプロセスのPIDを自動判定することができない
                * メインプロセスのPIDを記載したPIDファイルを生成する
                * これを「PIDFile」に指定すると、systemdはこのファイルからPIDを認識できる
    * BusName
        * D-Bus型サービスのbus接続名
    * Restart
        * サービスプロセス停止時の再起動条件
        * デフォルトは「no」
    * PrivateTmp
        * サービス専用の/tmpと/var/tmpを用意する
* Installセクション
    * ユニットファイルの共通的な設定
        * 「systemctl enable/disable」コマンドの動作に影響する設定項目
    * WantedBy
        * enable時にこのUnitの.wantsディレクトリにリンクを作成する
    * RequiredBy
        * enable時にこのUnitの.requiredディレクトリにリンクを作成する
    * Also
        * enable/disable時に同時にenable/disableするUnit
    * Alias
        * enable時にこのUnitの別名を用意
        
### 特殊環境変数

* `$MAINPID`
    * systemdから実行したコマンドのPID
        * メインプロセスのPIDとして認識される
        * 「ExecReload」に対してメインプロセスにHUPシグナルを送るコマンドを記載する際になどに使用できる

### D-Bus

* D-Busとは
    * systemdの環境で標準となるプロセス間通信用のメッセージバス機能
    * 起動後に固有の「bus接続名」を指定して、D-Busに接続を行う

### postfix

```
### /usr/lib/systemd/system/postfix.service
================================================
[Unit]
Description=Postfix Mail Transport Agent
After=syslog.target network.target
Conflicts=sendmail.service exim.service

[Service]
Type=forking
PIDFile=/var/spool/postfix/pid/master.pid
EnvironmentFile=-/etc/sysconfig/network
ExecStartPre=-/usr/libexec/postfix/aliasesdb
ExecStartPre=-/usr/libexec/postfix/chroot-update
ExecStart=/usr/sbin/postfix start
ExecReload=/usr/sbin/postfix reload
ExecStop=/usr/sbin/postfix stop

[Install]
WantedBy=multi-user.target
================================================
```

* Unitセクション
    * Conflicts
        * Systemdはこの関係が定義されたプロセスは同時起動しない
* Serviceセクション
    * イコール「=」直後のハイフン「-」の意味
        * EnvironmentFile
            * ファイルが存在しなければ読み込まないだけで、エラーになったり警告を出したりもしない
        * ExecStartPre
            * コマンドが失敗してもエラーにならない
* Installセクション
    * WantedBy=multi-user.target
        * 「multi-user.target.wants」ディレクトリ内にこの設定ファイルへのシンボリックリンクが作成される
        * 一般的なサービスの場合
            * multi-user.target（旧来のrunlevel 3に相当）、もしくはgraphical.target（runlevel 5に相当）

### shirasagi-unicorn

[shirasagi/bin/install.sh](https://github.com/shirasagi/shirasagi/blob/master/bin/install.sh#L303)

```
### /etc/systemd/system/shirasagi-unicorn.service
================================================
[Unit]
Description=Shirasagi Unicorn Server
After=mongod.service

[Service]
EnvironmentFile=-/etc/profile.d/rvm.sh
User=shirasagi
WorkingDirectory=/var/www/shirasagi
ExecStart=/usr/local/rvm/default/bundle exec rake unicorn:start
ExecStop=/usr/local/rvm/default/bundle exec rake unicorn:stop
ExecReload=/usr/local/rvm/default/bundle exec rake unicorn:restart
Type=forking
PIDFile=/var/www/shirasagi/tmp/pids/unicorn.pid

[Install]
WantedBy=multi-user.target
================================================
```

* Serviceセクション
    * User
        * 実行ユーザを指定
    * Group
        * グループを指定 (このファイルには無いがグループを指定できる)